In [1]:
import requests
import json
import pandas as pd
import datetime
from dateutil.relativedelta import *

In [2]:
with open('key_info.json') as json_file:
    json_data = json.load(json_file)
    key = json_data['key']

In [3]:
df = pd.read_csv('movie_data.csv')['movieCd']

In [4]:
def get_movie_info_string(data, info_type, info_detail):
    result = []
    for i in range(len(data[info_type])):
        result.append(data[info_type][i][info_detail])
    result = ','.join(result)
    return result

In [5]:
data_list = []
for movieCd in df:
    data_dict = {}
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key='+key+'&movieCd='+str(movieCd)
    data = json.loads(requests.get(url).text)['movieInfoResult']['movieInfo']
    data_dict['movieCd']      = [data['movieCd']]
    data_dict['movieNmEn']    = [data['movieNmEn']]
    data_dict['showTm']       = [data['showTm']]
    data_dict['typeNm']       = [data['typeNm']]
    data_dict['nationNm']     = [get_movie_info_string(data, 'nations', 'nationNm')]
    data_dict['genreNm']      = [get_movie_info_string(data, 'genres', 'genreNm')]
    data_dict['directors']    = [get_movie_info_string(data, 'directors', 'peopleNm')]
    data_dict['actors']       = [get_movie_info_string(data, 'actors', 'peopleNm')]
    data_dict['watchGradeNm'] = [get_movie_info_string(data, 'audits', 'watchGradeNm')]
    data_df = pd.DataFrame(data_dict)
    data_list.append(data_df)

In [6]:
all_df = pd.concat(data_list).reset_index(drop=True)
all_df.head()

,movieCd,movieNmEn,showTm,typeNm,nationNm,genreNm,directors,actors,watchGradeNm
0,20201122,Tenet,149,장편,미국,"액션,SF",크리스토퍼 놀란,"존 데이비드 워싱턴,로버트 패틴슨,엘리자베스 데비키,애런 존슨,마이클 케인,케네스 ...",12세이상관람가
1,20197922,DELIVER US FROM EVIL,107,장편,한국,"범죄,액션",홍원찬,"황정민,이정재,박정민",15세이상관람가


In [4]:
all_df.to_csv('movie_info_data.csv', index=False)

0       20201122
1       20197922
2       20199883
3       20202281
4       20010238
          ...   
1195    20100305
1196    20100214
1197    20100814
1198    20100810
1199    20100813
Name: movieCd, Length: 1200, dtype: int64